# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (20, 1000)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 20 to 1000:

Moe_Szyslak: Looks like this is the end.
Barney_Gumble: That's all right. I couldn't have led a richer life.
Barney_Gumble: So the next time somebody tells you county folk are good, honest people, you can spit in their faces for me!
Lisa_Simpson: I will, Mr. Gumbel. But if you'll excuse me, I'm profiling my dad for the school paper. I thought it would be neat to follow him around for a day to see what makes him tick.
Barney_Gumble: Oh, that's sweet. I used to follow my dad to a lot of bars too. (BELCH)
Moe_Szyslak: Here you go. One beer, one chocolate milk.
Lisa_Simpson: Uh, excuse me, I have the chocolate milk.
Moe_Szyslak: Oh.
Moe_Szyslak: What's the matter, Homer? The depressin' effects of alcohol usually don't kick in 'til closing time.
Lis

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = sorted(set(text))
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))

    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [4]:
test_text = '''
        Moe_Szyslak Moe's Tavern Where the elite meet to drink
        Bart_Simpson Eh yeah hello is Mike there Last name Rotch
        Moe_Szyslak Hold on I'll check Mike Rotch Mike Rotch Hey has anybody seen Mike Rotch lately
        Moe_Szyslak Listen you little puke One of these days I'm gonna catch you and I'm gonna carve my name on your back with an ice pick
        Moe_Szyslak Whats the matter Homer You're not your normal effervescent self
        Homer_Simpson I got my problems Moe Give me another one
        Moe_Szyslak Homer hey you should not drink to forget your problems
        Barney_Gumble Yeah you should only drink to enhance your social skills'''

# create_lookup_tables(text)
print(create_lookup_tables(test_text))

({'\n': 0, ' ': 1, "'": 2, 'B': 3, 'E': 4, 'G': 5, 'H': 6, 'I': 7, 'L': 8, 'M': 9, 'O': 10, 'R': 11, 'S': 12, 'T': 13, 'W': 14, 'Y': 15, '_': 16, 'a': 17, 'b': 18, 'c': 19, 'd': 20, 'e': 21, 'f': 22, 'g': 23, 'h': 24, 'i': 25, 'k': 26, 'l': 27, 'm': 28, 'n': 29, 'o': 30, 'p': 31, 'r': 32, 's': 33, 't': 34, 'u': 35, 'v': 36, 'w': 37, 'y': 38, 'z': 39}, {0: '\n', 1: ' ', 2: "'", 3: 'B', 4: 'E', 5: 'G', 6: 'H', 7: 'I', 8: 'L', 9: 'M', 10: 'O', 11: 'R', 12: 'S', 13: 'T', 14: 'W', 15: 'Y', 16: '_', 17: 'a', 18: 'b', 19: 'c', 20: 'd', 21: 'e', 22: 'f', 23: 'g', 24: 'h', 25: 'i', 26: 'k', 27: 'l', 28: 'm', 29: 'n', 30: 'o', 31: 'p', 32: 'r', 33: 's', 34: 't', 35: 'u', 36: 'v', 37: 'w', 38: 'y', 39: 'z'})


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    return {
        '.':  '||DOT||',
        ',':  '||COMMA||',
        '"':  '||QUOTATION_MARK||',
        ';':  '||SEMICOLON||',
        '!':  '||EXCLAMATION_MARK||',
        '?':  '||QUESTION_MARK||',
        '(':  '||LEFT_PARENTHESES||',
        ')':  '||RIGHT_PARENTHESES||',
        '--': '||DASH||',
        '\n': '||RETURN||'
    }

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


In [6]:
print(token_lookup()['.'])

||DOT||


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/Users/danielwillemse/anaconda3/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [51]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    Input = tf.placeholder(tf.int32, (None, None),name='input')
    Targets = tf.placeholder(tf.int32, (None, None), name='targets')
    LearningRate = tf.placeholder(tf.float32, name='learning_rate')
    
    return Input, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [52]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # I could clean the code up a bit by not initialising a lstm_layers variable and
    # just building one straight away. But this leaves me easy access to play around a bit
    lstm_layers = 1
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * lstm_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    return cell, tf.identity(initial_state, name='initial_state')


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [53]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [54]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    Outputs, FinalState = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    return Outputs, tf.identity(FinalState, name='final_state')


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [55]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    Outputs, FinalState = build_rnn(cell, get_embed(input_data, vocab_size, embed_dim))
    Logits = tf.contrib.layers.fully_connected(Outputs, vocab_size, 
                                               activation_fn=None,
                                               weights_initializer=tf.truncated_normal_initializer(),
                                               biases_initializer=tf.zeros_initializer(),
                                               trainable=True
                                              )
    return Logits, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  [
    [[ 1  2], [ 7  8], [13 14]]
    [[ 2  3], [ 8  9], [14 15]]
  ]

  [
    [[ 3  4], [ 9 10], [15 16]]
    [[ 4  5], [10 11], [16 17]]
  ]

  [
    [[ 5  6], [11 12], [17 18]]
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [56]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(xdata)
    ydata[0:-1], ydata[-1] = xdata[1:],xdata[0]

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

    return np.array(list(zip(x_batches, y_batches)))
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [70]:
num_epochs = 100
batch_size = 256 # 32 - 512
rnn_size = 256
embed_dim = 256
seq_length = 10
learning_rate = 0.001 # 0.1 - 0.0001
show_every_n_batches = 4

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [71]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [72]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/26   train_loss = 9.889
Epoch   0 Batch    4/26   train_loss = 8.436
Epoch   0 Batch    8/26   train_loss = 7.956
Epoch   0 Batch   12/26   train_loss = 7.637
Epoch   0 Batch   16/26   train_loss = 7.225
Epoch   0 Batch   20/26   train_loss = 7.102
Epoch   0 Batch   24/26   train_loss = 6.875
Epoch   1 Batch    2/26   train_loss = 6.597
Epoch   1 Batch    6/26   train_loss = 6.491
Epoch   1 Batch   10/26   train_loss = 6.268
Epoch   1 Batch   14/26   train_loss = 6.361
Epoch   1 Batch   18/26   train_loss = 6.208
Epoch   1 Batch   22/26   train_loss = 6.051
Epoch   2 Batch    0/26   train_loss = 5.937
Epoch   2 Batch    4/26   train_loss = 5.736
Epoch   2 Batch    8/26   train_loss = 5.760
Epoch   2 Batch   12/26   train_loss = 5.797
Epoch   2 Batch   16/26   train_loss = 5.614
Epoch   2 Batch   20/26   train_loss = 5.662
Epoch   2 Batch   24/26   train_loss = 5.579
Epoch   3 Batch    2/26   train_loss = 5.469
Epoch   3 Batch    6/26   train_loss = 5.444
Epoch   3 

Epoch  28 Batch    4/26   train_loss = 1.693
Epoch  28 Batch    8/26   train_loss = 1.640
Epoch  28 Batch   12/26   train_loss = 1.691
Epoch  28 Batch   16/26   train_loss = 1.645
Epoch  28 Batch   20/26   train_loss = 1.561
Epoch  28 Batch   24/26   train_loss = 1.621
Epoch  29 Batch    2/26   train_loss = 1.547
Epoch  29 Batch    6/26   train_loss = 1.598
Epoch  29 Batch   10/26   train_loss = 1.643
Epoch  29 Batch   14/26   train_loss = 1.590
Epoch  29 Batch   18/26   train_loss = 1.544
Epoch  29 Batch   22/26   train_loss = 1.505
Epoch  30 Batch    0/26   train_loss = 1.422
Epoch  30 Batch    4/26   train_loss = 1.512
Epoch  30 Batch    8/26   train_loss = 1.488
Epoch  30 Batch   12/26   train_loss = 1.496
Epoch  30 Batch   16/26   train_loss = 1.513
Epoch  30 Batch   20/26   train_loss = 1.429
Epoch  30 Batch   24/26   train_loss = 1.486
Epoch  31 Batch    2/26   train_loss = 1.430
Epoch  31 Batch    6/26   train_loss = 1.452
Epoch  31 Batch   10/26   train_loss = 1.462
Epoch  31 

Epoch  56 Batch    8/26   train_loss = 0.518
Epoch  56 Batch   12/26   train_loss = 0.522
Epoch  56 Batch   16/26   train_loss = 0.536
Epoch  56 Batch   20/26   train_loss = 0.520
Epoch  56 Batch   24/26   train_loss = 0.513
Epoch  57 Batch    2/26   train_loss = 0.515
Epoch  57 Batch    6/26   train_loss = 0.530
Epoch  57 Batch   10/26   train_loss = 0.505
Epoch  57 Batch   14/26   train_loss = 0.527
Epoch  57 Batch   18/26   train_loss = 0.496
Epoch  57 Batch   22/26   train_loss = 0.504
Epoch  58 Batch    0/26   train_loss = 0.471
Epoch  58 Batch    4/26   train_loss = 0.513
Epoch  58 Batch    8/26   train_loss = 0.502
Epoch  58 Batch   12/26   train_loss = 0.501
Epoch  58 Batch   16/26   train_loss = 0.525
Epoch  58 Batch   20/26   train_loss = 0.499
Epoch  58 Batch   24/26   train_loss = 0.500
Epoch  59 Batch    2/26   train_loss = 0.501
Epoch  59 Batch    6/26   train_loss = 0.515
Epoch  59 Batch   10/26   train_loss = 0.491
Epoch  59 Batch   14/26   train_loss = 0.504
Epoch  59 

Epoch  84 Batch   12/26   train_loss = 0.390
Epoch  84 Batch   16/26   train_loss = 0.405
Epoch  84 Batch   20/26   train_loss = 0.394
Epoch  84 Batch   24/26   train_loss = 0.389
Epoch  85 Batch    2/26   train_loss = 0.402
Epoch  85 Batch    6/26   train_loss = 0.409
Epoch  85 Batch   10/26   train_loss = 0.380
Epoch  85 Batch   14/26   train_loss = 0.406
Epoch  85 Batch   18/26   train_loss = 0.389
Epoch  85 Batch   22/26   train_loss = 0.385
Epoch  86 Batch    0/26   train_loss = 0.354
Epoch  86 Batch    4/26   train_loss = 0.393
Epoch  86 Batch    8/26   train_loss = 0.379
Epoch  86 Batch   12/26   train_loss = 0.387
Epoch  86 Batch   16/26   train_loss = 0.402
Epoch  86 Batch   20/26   train_loss = 0.392
Epoch  86 Batch   24/26   train_loss = 0.386
Epoch  87 Batch    2/26   train_loss = 0.400
Epoch  87 Batch    6/26   train_loss = 0.406
Epoch  87 Batch   10/26   train_loss = 0.377
Epoch  87 Batch   14/26   train_loss = 0.405
Epoch  87 Batch   18/26   train_loss = 0.387
Epoch  87 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [73]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [76]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [79]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_0 = loaded_graph.get_tensor_by_name('input:0')
    initial_state_0 = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state_0 = loaded_graph.get_tensor_by_name('final_state:0')
    probs_0 = loaded_graph.get_tensor_by_name('probs:0')
    return input_0, initial_state_0, final_state_0, probs_0


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [85]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    
    return int_to_vocab[np.argmax(probabilities)]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [86]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(into phone) gotcha ya down for forty bucks.


moe_szyslak: think, homer, this ain't tryin' to you've got an face it.
marge_simpson: a, shouldn't, check.
moe_szyslak: well, you went to not the love of that.
homer_simpson:(intrigued) or do i do?
moe_szyslak:...
homer_simpson: the call is a from time...
moe_szyslak: get me! i just wanted it.
moe_szyslak:(too beer) sure thing you got all of that three usually sir.
artie_ziff: that three times.
.
moe_szyslak: and i'm trying to mention that you did in the sink.
homer_simpson:(eyes have have) no me here?
c. are. what don't blame. i could stay there!
homer_simpson:(laughs) now if you want to be the letter.
marge_simpson: oh, i'm very big voice.
mr... who the hell's stillwater: so i stayed up behind get him with? get 'em? huh? listen on edner.


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.